In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# 데이터마트용 계정 생성
# sqlDeveloper에서 진행
# create user dm_admin IDENTIFIED BY "123qwe!@#QWE";
# grant dwrole to dm_admin;
# grant unlimited tablespace to dm_admin;


In [1]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt



In [3]:
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    rc('font', family='NanumGothic')

In [2]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

### JDBC

In [3]:
JDBC_DW = {
    'url':'jdbc:oracle:thin:@decorona_high?TNS_ADMIN=/home/big/study/db/Wallet_DECORONA'
    ,'props':{
        'user':'dw_de'
       ,'password':'123qwe!@#QWE'
    }   
}

JDBC_DM = {
    'url':'jdbc:oracle:thin:@decorona_high?TNS_ADMIN=/home/big/study/db/Wallet_DECORONA'
    ,'props':{
        'user':'dm_de'
       ,'password':'123qwe!@#QWE'
    }   
}

## Load DataMart

### 1. 지역별 단위면적(km)당 인구수와 코로나 확진자 수 데이터 저장

In [5]:
popu = spark.read.jdbc(url=JDBC_DW['url'], table='loc', properties=JDBC_DW['props'])
patients = spark.read.jdbc(url=JDBC_DW['url'], table='corona_patients', properties=JDBC_DW['props'])
popu.show(3)
patients.show(3)

+----+----+----------+------------+
| LOC|AREA|POPULATION|FACILITY_CNT|
+----+----+----------+------------+
|서울| 605|   9736027|        5224|
|부산| 770|   3396109|        1462|
|대구| 883|   2412642|        1015|
+----+----+----------+------------+
only showing top 3 rows



+------+----+---------+-------+-----------+--------+----------+
|cp_idx| LOC|DEATH_CNT|DEF_CNT|LOC_OCC_CNT|QUR_RATE|   STD_DAY|
+------+----+---------+-------+-----------+--------+----------+
|     1|세종|       43| 191189|        206|   51409|2022-09-19|
|     2|제주|      217| 331125|        218|   48928|2022-09-19|
|     3|부산|     2423|1456840|        633|   43483|2022-09-19|
+------+----+---------+-------+-----------+--------+----------+
only showing top 3 rows



In [6]:
pop_patients = popu.join(patients, on='loc') \
                    .select('loc'
                           ,ceil(col('population')/col('area')).alias('POPU_DENSITY')
                            ,'qur_rate'
                            ,'std_day') \
                    .orderBy(col('std_day'))

pop_patients.show(3)

+----+------------+--------+----------+
| loc|POPU_DENSITY|qur_rate|   std_day|
+----+------------+--------+----------+
|강원|          93|   46475|2022-09-19|
|경기|        1366|   48319|2022-09-19|
|경남|         321|   44509|2022-09-19|
+----+------------+--------+----------+
only showing top 3 rows



In [10]:
pop_patients.write.jdbc(url=JDBC_DM['url'],  table='CO_POPU_DENSITY', mode='append', properties=JDBC_DM['props'])

### 2.  백신접종 완료자와 코로나 확진자 수 데이터 저장

In [19]:
vaccine = spark.read.jdbc(url=JDBC_DW['url'], table='CORONA_VACCINE', properties=JDBC_DW['props'])

In [12]:
vaccine.show(3)

+------+----+----+-------+----------+
|CV_IDX| LOC|V_TH|  V_CNT|   STD_DAY|
+------+----+----+-------+----------+
|     1|서울|  v1|8340594|2022-09-19|
|     2|서울|  v2|8261803|2022-09-19|
|     3|서울|  v3|6059101|2022-09-19|
+------+----+----+-------+----------+
only showing top 3 rows



In [24]:
pd_vaccine = vaccine.to_pandas_on_spark()
pd_vaccine = pd_vaccine.pivot_table(index=['LOC','STD_DAY'], columns='V_TH', values='V_CNT')
pd_vaccine = pd_vaccine.reset_index()
tmp = pd_vaccine.to_spark()

vac_rate = tmp.join(popu, on='LOC') \
        .join(patients, on='LOC') \
        .select('LOC', patients.STD_DAY
                ,ceil(col('v1')/col('population') * 100).alias('V1TH_RATE')
                ,ceil(col('v2')/col('population') * 100).alias('V2TH_RATE')
                ,ceil(col('v3')/col('population') * 100).alias('V3TH_RATE')
                ,ceil(col('v4')/col('population') * 100).alias('V4TH_RATE')
                ,'QUR_RATE'
               ) \
vac_rate.show(3)


+----+----------+---------+---------+---------+---------+--------+
| LOC|   STD_DAY|V1TH_RATE|V2TH_RATE|V3TH_RATE|V4TH_RATE|QUR_RATE|
+----+----------+---------+---------+---------+---------+--------+
|강원|2022-09-19|       87|       87|       69|       18|   46475|
|경기|2022-09-19|       85|       85|       63|       13|   48319|
|경남|2022-09-19|       86|       85|       64|       15|   44509|
+----+----------+---------+---------+---------+---------+--------+
only showing top 3 rows



In [25]:
vac_rate.write.jdbc(url=JDBC_DM['url'],  table='CO_VACCINE', mode='append', properties=JDBC_DM['props'])

### 3. 다중이용시설과 코로나 확진자 수의 상관관계

### 4. 요일별 코로나 확진자 수를 구해보자